# Lynden-Bell & Pringle Test

In [ ]:
import twopoppy2
import numpy as np
import matplotlib.pyplot as plt
from twopoppy2.constants import year, au, k_b, G, m_p, year, R_sun
from twopoppy2.utils import lbp_solution

%matplotlib inline

In [ ]:
m = twopoppy2.Twopoppy(rmax=2000 * au)

sig0  = 15.766
T0    = 36.5
q     = 0.5       # play with this temperature profile to test different gamma values
r_c   = 30 * au
alpha = 1e-2
d2g   = 1e-2

In [ ]:
gamma = 1.5 - q
m.set_all_alpha(alpha)
m.T_gas = T0 * (m.r / r_c)**-q
m.sigma_g = sig0 * (m.r / r_c)**-gamma * np.exp(-(m.r/r_c)**(2-gamma))
m.sigma_d = d2g * m.sigma_g
M_disk = (np.pi * np.diff(m.ri**2) * m.sigma_g).sum()

In [ ]:
m.initialize()

In [ ]:
m.run()

In [ ]:
cs1 = m.cs[0]
om1 = m.omega[0]
nu1 = m.gas_viscosity[0]
siga_0, _ = lbp_solution(m.r, gamma, nu1, m.M_star, M_disk, r_c)
siga_1, _ = lbp_solution(m.r, gamma, nu1, m.M_star, M_disk, r_c, time=m.time)

In [ ]:
f, ax = plt.subplots()
ax.loglog(m.r / au, siga_0, 'k', label='analytic solution')
ax.loglog(m.r / au, siga_1, 'k')
ax.loglog(m.r / au, m.data['sigma_g'][0], '--', label='code @ {:0.3g} yr'.format(m.data['time'][0][0] / year))
ax.loglog(m.r / au, m.data['sigma_g'][-1], '--', label='code @ {:0.3g} yr'.format(m.data['time'][-1][0] / year))
ax.set_ylim(1e-4, 1e5)
ax.set_xlabel('radius [au]')
ax.set_ylabel(r'$\Sigma_\mathrm{g}$ [g cm$^{-2}$]')
ax.set_title(f'$\gamma = {gamma:.2g}$')
ax.legend();